# Code

## Different encoding techniques

### Bag of words
* Transforms words into numbers using a dictionay.
* We only care about the frequency of words in a sentence.
* Bad because it doesn't take into account the ordering of those words!

In [1]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


### Word embeddings
* Tranforms words into a N dimensional vector.
* Tries to make similar words point to similar directions and antonym words in opposite directions.
* It's a layer of the model.

## Setiment Analysis with Recurrent Neural Networks

In [13]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 400
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [14]:
train_data.shape

(25000,)

In [15]:
len(train_data[3])

550

Padding reviews so that they can be the same size.

In [16]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [17]:
len(train_data[3])

400

### Creating the model

In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 16),
    tf.keras.layers.LSTM(16),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 16)          1417344   
                                                                 
 lstm_2 (LSTM)               (None, 16)                2112      
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,419,473
Trainable params: 1,419,473
Non-trainable params: 0
_________________________________________________________________


### Training

In [31]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 13s 18ms/step - loss: 0.4440 - acc: 0.7921 - val_loss: 0.3149 - val_acc: 0.8718
Epoch 2/10
625/625 [==============================] - 11s 17ms/step - loss: 0.2421 - acc: 0.9089 - val_loss: 0.2976 - val_acc: 0.8890
Epoch 3/10
625/625 [==============================] - 11s 17ms/step - loss: 0.1801 - acc: 0.9352 - val_loss: 0.2776 - val_acc: 0.8888
Epoch 4/10
625/625 [==============================] - 11s 18ms/step - loss: 0.1457 - acc: 0.9477 - val_loss: 0.2797 - val_acc: 0.8980
Epoch 5/10
625/625 [==============================] - 11s 18ms/step - loss: 0.1216 - acc: 0.9579 - val_loss: 0.3711 - val_acc: 0.8762
Epoch 6/10
625/625 [==============================] - 12s 19ms/step - loss: 0.1086 - acc: 0.9628 - val_loss: 0.3818 - val_acc: 0.8886
Epoch 7/10
625/625 [==============================] - 11s 18ms/step - loss: 0.0942 - acc: 0.9689 - val_loss: 0.3594 - val_acc: 0.8880
Epoch 8/10
625/625 [==============================] - 11s 18ms

In [32]:
model.evaluate(test_data, test_labels)

782/782 [==============================] - 7s 8ms/step - loss: 0.4251 - acc: 0.8573


[0.4250611960887909, 0.8572800159454346]

### Predicting

In [33]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


1654784/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [34]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [42]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,MAXLEN))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

review = "really nice movie nice nice nice nice nice nice nice nice nice nice"
predict(review)

[0.8909225]
[0.277831]
[0.08351763]
